In [1]:
import pandas as pd
import numpy as np
import bqplot
import ipywidgets
import matplotlib.pyplot as plt
from bqplot import market_map
import traitlets
from IPython.display import clear_output, display
from knapsack_problem_solver import *

In [2]:
trucks = [transit, e350, box_truck, platform]
branches = ['ZAGA']

In [3]:
branch_options = ipywidgets.Dropdown(
    options = branches,
    description = 'Center: '
)
truck_options = ipywidgets.Dropdown(
    options = trucks,
    description = 'Truck: '
)
style = {'description_width': 'initial'}
empty_len = ipywidgets.FloatSlider(
    value = 0.5,
    min = 0,
    max = 1,
    step = 0.05,
    description = 'Empty Length: ',
    style = style
)
empty_wid = ipywidgets.FloatSlider(
    value = 0.5,
    min = 0,
    max = 1,
    step = 0.05,
    description = 'Empty Width: ',
    style = style
)
empty_hei = ipywidgets.FloatSlider(
    value = 0.5,
    min = 0,
    max = 1,
    step = 0.05,
    description = 'Empty Height: ',
    style = style
)
title = ipywidgets.HTML(' \
                       <!DOCTYPE html> \
                        <html> \
                        <head> \
                        <style> \
                        .center { \
                            margin: 0 auto; \
                            max-width: 50%; \
                            padding: 1.3rem; \
                        } \
 \
                        .title-bar { \
                            background-color: #8497b0; \
                            color: #ffffff; \
                        } \
 \
                        </style> \
                        </head> \
                        <body> \
                        <div class="title-bar"> \
                            <div class="center"> \
                              <h3>Truck Distribution Recommendation System</h3> \
                            </div> \
                        </div> \
                        </body> \
                        </html> \
                     ')

In [4]:
theme_blue = '#8497b0'

layout_center_list = ipywidgets.Layout()
layout_center_list.width = '50%'
layout_center_list.margin= '0.1% auto'

layout_center_subtitle = ipywidgets.Layout()
layout_center_subtitle.width = '50%'
layout_center_subtitle.margin= '2% auto'

whole_layout_center = ipywidgets.Layout()
whole_layout_center.width = '100%'
whole_layout_center.margin = '1% auto'

layout_button = ipywidgets.Layout()
layout_button.width = '50%'
layout_button.margin = '1% auto'

two_opposite_side = ipywidgets.Layout()
two_opposite_side.display = 'flex'
two_opposite_side.justify_content = 'space-between'

add_top_margin = ipywidgets.Layout()
add_top_margin.margin = '5% 0% 0.1%'

In [5]:
button = ipywidgets.Button(description='Calculate', button_style='success')
output = ipywidgets.Output()
sol_out = ipywidgets.Output()
data_out = ipywidgets.Output()
input_df = None
sol_df = None
pie_chart_colors = [theme_blue, '#85c5a6', '#cbebb5']

In [6]:
def on_dataView_clicked(b):
    with data_out:
        clear_output(True)
        pie_candidate_class_distribution = ipywidgets.Output()
        pie_space_taken_class = ipywidgets.Output()
        pie_profit_contribution_class = ipywidgets.Output()
        
        with pie_candidate_class_distribution:
            # candidate list material class distribution pie chart
            clear_output(True)
            labels = list(set(input_df['Material Class']))
            class_size = input_df.groupby('Material Class')['Material Class'].size()
            sizes = [class_size[label] for label in labels]
            fig1, ax1 = plt.subplots()
            ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
                    shadow=True, startangle=0, colors=pie_chart_colors)
            ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
            ax1.set_title('Candidate Products Distribution by Material Class')
            plt.show()
            
        with pie_space_taken_class:
            # algo solution material class space taken pie chart
            clear_output(True)
            labels = list(set(sol_df['Material Class']))
            sizes = []
            for label in labels:
                materials = list(sol_df[sol_df['Material Class'] == label]['GBID'])
                space = 0
                for gbid in materials:
                    row = input_df[input_df['GBID'] == gbid]
                    space += float(row['Length']) * float(row['Width']) * float(row['Height'])
                sizes.append(space)
            fig1, ax1 = plt.subplots()
            ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
                    shadow=True, startangle=0, colors=pie_chart_colors)
            ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
            ax1.set_title('Solution: Space Taken by Material Class')
            plt.show()
        
        with pie_profit_contribution_class:
            # algo solution material class conotribution of profit pie chart
            clear_output(True)
            labels = list(set(sol_df['Material Class']))
            sizes = []
            for label in labels:
                tmp_df = sol_df[sol_df['Material Class'] == label].reset_index()
                profit = 0
                for i in range(len(tmp_df)):
                    profit += tmp_df.iloc[i]['Quantity'] * tmp_df.iloc[i]['Unit Profit']
                sizes.append(profit)
            fig1, ax1 = plt.subplots()
            ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
                    shadow=True, startangle=0, colors=pie_chart_colors)
            ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
            ax1.set_title('Solution: Profit Contribution by Material Class')
            plt.show()
        
        original_sol_pie = ipywidgets.HBox([pie_candidate_class_distribution, pie_space_taken_class, pie_profit_contribution_class])
        original_sol_pie.layout = two_opposite_side
        
        data_view_title = ipywidgets.HTML(
            value =  '<div align="center" style="border:1px solid {0}"> \
                <h4 style="color: black">Visualizations for Algorithm Solution</h4> \
            </div>'.format(theme_blue)
        )
        data_view_title.layout = layout_center_subtitle
        
        data_view = ipywidgets.VBox([data_view_title, original_sol_pie])
        display(data_view)

In [7]:
def on_button_clicked(b):
    with output:
        # clear all previous output display
        clear_output(True)
        global data_out
        data_out = ipywidgets.Output()
        data_out.layout = whole_layout_center
        
        intermediate_out = ipywidgets.Output()
        intermediate_out.layout = layout_center_list
        with intermediate_out:
            display(ipywidgets.HTML(
                value = '<h4>Calculating Recommendation...</h4>'
            ))

            solver = KnapsackSolver(KnapsackSolverParser(), truck_options.value, [empty_len.value, empty_wid.value, empty_hei.value], True)
            transformer = solver.transformer

            global input_df
            input_df = transformer.df
            input_df = input_df[['Material', 'Material Class', 'GMR for Reporting', 'Unit_profit', 'Length', 'Width', 'Height']]
            input_df = input_df.rename(columns={'GMR for Reporting': 'GMR', 'Unit_profit': 'Unit Profit', 'Material': 'GBID'})
            display(input_df)
        display(intermediate_out)
        
        best_value, best_sol = solver.solve()
        global sol_df
        sol_df = transformer.transform_solution(best_sol)
        
        clear_output(True)
        
        value_display = ipywidgets.HTML(
            value = '<div align="center" style="border:1px solid {1}"> \
                <h4 style="color: black">Profit when 100% sold: <b>${0}</b></h4> \
            </div>'.format(best_value, theme_blue)
        )
        value_display.layout = layout_center_list
        
        intro_sol_out = ipywidgets.HTML(
            value = '<h4>Recommendation of products : </h4>'
        )
        intro_sol_out.layout = add_top_margin
        
        show_pie_profit_contribution_class_button = ipywidgets.Button(description='Data Analytics View', button_style='success')
        show_pie_profit_contribution_class_button.on_click(on_dataView_clicked)
        show_pie_profit_contribution_class_button.style.button_color = theme_blue
        show_pie_profit_contribution_class_button.layout = add_top_margin
        
        with sol_out:
            clear_output(True)
            display(sol_df)
            
        intro_line = ipywidgets.HBox([ipywidgets.VBox([intro_sol_out, sol_out]), show_pie_profit_contribution_class_button])
        intro_line.layout = two_opposite_side
        
        regular_sol_display = ipywidgets.VBox([
            value_display, 
            intro_line
        ])
        regular_sol_display.layout = whole_layout_center
            
        display(ipywidgets.VBox([
            regular_sol_display,
            data_out
        ]))

button.on_click(on_button_clicked)

spaces_box = ipywidgets.VBox([empty_len, empty_wid, empty_hei])
user_operation_box = ipywidgets.VBox([branch_options, truck_options, spaces_box, button])

branch_options.layout = layout_center_list
truck_options.layout = layout_center_list
empty_hei.layout = layout_center_list
empty_len.layout = layout_center_list
empty_wid.layout = layout_center_list

button.style.button_color  = theme_blue
button.layout = layout_button

display(ipywidgets.VBox([title, user_operation_box, output]))